In [29]:
import csv
import numpy as np 

In [30]:
###  Load data

def read_csv(file):
    with open(file, newline='') as f:
        reader = csv.reader(f)
        line_count = 0
        rows = []
        for row in reader:
            if line_count == 0:
                titles = row
            else:
                rows.append(row)
            line_count += 1
    rows_int = np.array([[int(r) for r in row] for row in rows])
    return titles, rows_int
        

titles, rows_train = read_csv('data/studentsdigits-train.csv')
assert titles[-1] == 'Digit' and len(titles) == 9, 'Not train set'
X_train = rows_train[:,0:len(titles)-1]
Y_train = rows_train[:,-1]
print(X_train.shape)
print(Y_train.shape)

titles, rows_test = read_csv('data/studentsdigits-test.csv')
assert len(titles) == 8, 'Not test set'
X_test = rows_test
print(X_test.shape)

(3747, 8)
(3747,)
(3747, 8)


In [31]:
import torch
from torch import nn
from sklearn.model_selection import KFold
from collections import OrderedDict
from sklearn.metrics import accuracy_score
from tqdm import tqdm, trange


def sanitize_param_name(param_name):
    return (
        param_name.replace("(", "")
        .replace(")", "")
        .replace(",", "_")
        .replace(" ", "_")
        .replace("<", "")
        .replace(">", "")
        .replace("'", "")
    )

In [32]:
# Build the MLP model


import time


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        torch.manual_seed(0)  # reproducibility

        # self.layers = nn.ModuleDict(
        #     OrderedDict(
        #         [
        #             ("fc_1", nn.Linear(in_features=8, out_features=32)),
        #             ("relu_1", nn.ReLU()),
        #             ("fc_2", nn.Linear(in_features=32, out_features=16)),
        #             ("relu_2", nn.ReLU()),
        #             ("fc_3", nn.Linear(in_features=16, out_features=10)),
        #             ("relu_3", nn.ReLU()),
        #         ]
        #     )
        # )
        # Maybe a little bit deepere rnn
        self.rnn = nn.RNN(input_size=2, hidden_size=16, num_layers=4, batch_first=True)
        self.fc = nn.Linear(in_features=16, out_features=10)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.lossFunction = nn.CrossEntropyLoss()

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])
        return x

    def run(self, X_train, Y_train):
        kf = KFold(n_splits=10, shuffle=False)
        # k-fold cross validation
        for fold, (train_index, test_index) in enumerate(
            tqdm(kf.split(X_train), total=kf.get_n_splits())
        ):
            self.train()
            x_train_fold, x_evaluate_fold = X_train[train_index], X_train[test_index]
            y_train_fold, y_evaluate_fold = Y_train[train_index], Y_train[test_index]

            x_train_foldTensor = torch.tensor(x_train_fold, dtype=torch.float32).view(
                -1, 4, 2
            )
            y_train_foldTensor = torch.tensor(y_train_fold, dtype=torch.long)
            x_evaluate_fold = torch.tensor(x_evaluate_fold, dtype=torch.float32).view(
                -1, 4, 2
            )
            y_evaluate_fold = torch.tensor(y_evaluate_fold, dtype=torch.long)

            for epoch in trange(600, desc=f"Fold {fold+1}", leave=False):
                self.optimizer.zero_grad()
                fold_pred = self.forward(x_train_foldTensor)
                loss = self.lossFunction(fold_pred, y_train_foldTensor)
                loss.backward()
                self.optimizer.step()

            # Evaluate using this fold
            self.eval()
            fold_evaluate_pred = self.forward(x_evaluate_fold)

            # Metrics
            acc = accuracy_score(y_evaluate_fold, fold_evaluate_pred.argmax(dim=1))
            lossItem = loss.item()
            print(f"Epoch: {epoch}, Loss: {lossItem}, Accuracy: {acc}")
        # Save the model
        torch.save(self.state_dict(), sanitize_param_name(f'./model-{time.strftime("%Y%m%d-%H%M%S")}.pth'))

    def predict(self, X_test):
        self.eval()
        x_testTensor = torch.tensor(X_test, dtype=torch.float32).view(-1, 4, 2)
        return self.forward(x_testTensor).argmax(dim=1)

In [33]:
# Train model
model = Model()
model.run(X_train, Y_train)

 10%|█         | 1/10 [00:05<00:49,  5.53s/it]

Epoch: 599, Loss: 0.22937868535518646, Accuracy: 0.0


 20%|██        | 2/10 [00:10<00:43,  5.42s/it]

Epoch: 599, Loss: 0.07700520753860474, Accuracy: 0.11466666666666667


 30%|███       | 3/10 [00:16<00:37,  5.36s/it]

...

Epoch: 599, Loss: 0.0056118411011993885, Accuracy: 0.9385026737967914


 90%|█████████ | 9/10 [00:48<00:05,  5.34s/it]

Epoch: 599, Loss: 0.004290617536753416, Accuracy: 0.9732620320855615


100%|██████████| 10/10 [00:53<00:00,  5.35s/it]

Epoch: 599, Loss: 0.0025399052537977695, Accuracy: 0.9331550802139037


In [36]:
### You code
prediction = model.predict(X_test)
print(prediction)
print(prediction.shape)

# Save prediction to text file and each line a number
# np.savetxt('upload_predictions.txt', prediction.numpy(), fmt='%d')
Y_test = prediction.numpy()

tensor([7, 1, 0,  ..., 5, 1, 7])
torch.Size([3747])


In [37]:
### Save prediction results
assert len(Y_test) == len(X_test), 'sizes dont match'
with open('upload_predictions.txt', 'w') as fp:
    fp.write('\n'.join(str(y) for y in Y_test))
print('SAVED')

SAVED
